In [1]:
# importing Relevant Packages
import numpy as np 
import pandas as pd 
import gc
import pickle
import itertools

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/jpx-my-baseline-skgb/__results__.html
/kaggle/input/jpx-my-baseline-skgb/simple_skgb.sav
/kaggle/input/jpx-my-baseline-skgb/submission.csv
/kaggle/input/jpx-my-baseline-skgb/__notebook__.ipynb
/kaggle/input/jpx-my-baseline-skgb/__output__.json
/kaggle/input/jpx-my-baseline-skgb/custom.css
/kaggle/input/jpx-my-baseline-lgboost/__results__.html
/kaggle/input/jpx-my-baseline-lgboost/simple_LGBM.txt
/kaggle/input/jpx-my-baseline-lgboost/simple_LGBM.sav
/kaggle/input/jpx-my-baseline-lgboost/submission.csv
/kaggle/input/jpx-my-baseline-lgboost/__notebook__.ipynb
/kaggle/input/jpx-my-baseline-lgboost/__output__.json
/kaggle/input/jpx-my-baseline-lgboost/custom.css
/kaggle/input/fork-jpx-dnn/dnn_2.tf.data-00000-of-00001
/kaggle/input/fork-jpx-dnn/dnn_1.tf.data-00000-of-00001
/kaggle/input/fork-jpx-dnn/dnn_2.tf.index
/kaggle/input/fork-jpx-dnn/__results__.html
/kaggle/input/fork-jpx-dnn/dnn_3.tf.data-00000-of-00001
/kaggle/input/fork-jpx-dnn/dnn_1.tf.index
/kaggle/input/fork-jpx-d

## Loading the Gradient Boosting Regression Models 

In [12]:
#loading models
model_1 = pickle.load(open('../input/jpx-my-baseline-xgboost/simple_xgb.sav', 'rb'))
model_2 = pickle.load(open('../input/jpx-my-baseline-lgboost/simple_LGBM.sav', 'rb'))
model_3 = pickle.load(open('../input/jpx-my-baseline-skgb/simple_skgb.sav', 'rb'))

#setting the wieghts:
w = [0.225,0.75,0.025] 


XGBoostError: [03:04:14] ../src/common/json.cc:458: Expecting: """, got: "76 ", around character position: 1
    {L\0\0\0\0\0\0\0
    ^~~~~~~~~
Stack trace:
  [bt] (0) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9133f) [0x7f6cbc71633f]
  [bt] (1) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0xc1090) [0x7f6cbc746090]
  [bt] (2) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0xc6c2e) [0x7f6cbc74bc2e]
  [bt] (3) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0xc1fcc) [0x7f6cbc746fcc]
  [bt] (4) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0xc208e) [0x7f6cbc74708e]
  [bt] (5) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x1d307e) [0x7f6cbc85807e]
  [bt] (6) /opt/conda/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterUnserializeFromBuffer+0x45) [0x7f6cbc704d05]
  [bt] (7) /opt/conda/lib/python3.7/lib-dynload/../../libffi.so.7(+0x69dd) [0x7f6d38cc09dd]
  

## Feature Engeneering

In [13]:
# define function to compute log returns
# def log_return(series, periods=1):
#     return np.log(series).diff(periods=periods)


# # def fill_the_gaps(df):
# #     new_df = pd.DataFrame(columns= df.columns)
# #     for i in range(len(df['SecuritiesCode'].unique())):
# #         new_df = new_df.append(df[df['SecuritiesCode'] == i].reindex(range(df[df['SecuritiesCode'] == i].index[0],df[df['SecuritiesCode'] == i].index[-1]+60,60),method='pad'))
# #     new_df.fillna(method = 'pad', inplace = True) 
# #     return new_df


# def rsiFunc(prices, n=14):
#     deltas = np.diff(prices)
#     seed = deltas[:n+1]
#     up = seed[seed>=0].sum()/n
#     down = -seed[seed<0].sum()/n
#     rs = up/down
#     rsi = np.zeros_like(prices)
#     rsi[:n] = 100. - 100./(1.+rs)

#     for i in range(n, len(prices)):
#         delta = deltas[i-1] # cause the diff is 1 shorter

#         if delta>0:
#             upval = delta
#             downval = 0.
#         else:
#             upval = 0.
#             downval = -delta

#         up = (up*(n-1) + upval)/n
#         down = (down*(n-1) + downval)/n

#         rs = up/down
#         rsi[i] = 100. - 100./(1.+rs)

#     return rsi



def get_features(df):

#     df['log_return_5'] = log_return(df['Close'],periods=5)
#     df['log_return'] = log_return(df['Close'],periods=1)

    upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
    lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    
#     df['EMA_21'] = df['Close'].ewm(span=21).mean()
    
#     df['EMA_55'] = df['Close'].ewm(span=55).mean()
    
#     df['EMA_315'] = df['Close'].ewm(span=315).mean()
    
#     df['EMA_825'] = df['Close'].ewm(span=825).mean()
    
#     window = 7
    
#     no_of_std = 2
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     window = 5
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     df['MACD'] = df['EMA_7'] - df['EMA_5']
    
    
#     df['rsi_5'] = rsiFunc(df['Close'].values, 5)
    
#     df['rsi_7'] = rsiFunc(df['Close'].values, 7)
    
#     df['rsi_21'] = rsiFunc(df['Close'].values, 21)
    
    
    
    df['VWAP'] = (df['Close'] * df['Volume'])/ df['Volume']




    
    
    #df = pd.concat([df, pd.get_dummies(df['Asset_ID'], prefix= 'Asset_')], axis=1)
    
    df[[ 'upper_shadow', 'lower_shadow','VWAP']].astype('float16')
    
    return df

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [15]:
scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']
features = ["SecuritiesCode",'Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']

## Building the DNN structure

In [16]:
# def setup_cv(df, splits=5):
#     df['fold'] = -1
#     df = df.sample(frac=1, random_state=42).reset_index(drop=True)
#     num_bins = int(np.floor(1 + np.log2(len(df))))
#     df.loc[:, "bins"] = pd.cut(
#         df["Target"], bins=num_bins, labels=False
#     )

#     kf = StratifiedKFold(n_splits=splits)
#     for f, (t_, v_) in enumerate(kf.split(X=df, y=df.bins.values)):
#             df.loc[v_, 'fold'] = f

#     df = df.drop("bins", axis=1)
#     return df

In [17]:
from sklearn.preprocessing import OrdinalEncoder
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [18]:

#codes = list(prices.SecuritiesCode.unique())
#codes_size = len(codes)


codes_size = 4416


def dense_block(x, units, act='swish', dr=0.2):
    x = L.Dropout(dr)(x)
    x = L.BatchNormalization()(x)
    x = L.Dense(units, activation=act)(x)
    return x

def get_dnn(dense_blocks):
    prices_in = L.Input(shape=(len(scaled_features),), name='input_prices')
    x_prices = L.BatchNormalization()(prices_in)
    x_prices = L.Dense(64, activation='swish')(x_prices)
    
    security_code_input = L.Input(shape=(1,), name='input_security_code')
    x_id = L.Embedding(codes_size, 32, input_length=1)(security_code_input)
    x_id = L.Reshape((-1, ))(x_id)
    x_id = L.Dense(32, activation='swish')(x_id)

    x = L.Concatenate(axis=1)([x_id, x_prices])
    
    for units in dense_blocks:
        x = dense_block(x, units)
    
    output = L.Dense(1)(x)
    
    model = M.Model([prices_in, security_code_input], 
                    [output])

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
                  loss='mse', metrics=['mse'])
    
    return model


In [19]:
# Define a simple sequential model
model = get_dnn([128, 64, 32])
# Display the model's architecture
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_security_code (InputLayer [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 32)        141312      input_security_code[0][0]        
__________________________________________________________________________________________________
input_prices (InputLayer)       [(None, 8)]          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 32)           0           embedding_1[0][0]                
____________________________________________________________________________________________

### Loading DNN weights

In [20]:
models = list()
for i in range(1,3):
    model.load_weights(f'../input/fork-jpx-dnn/dnn_{i}.tf')
    models.append(model)

## Submiting

In [21]:
#Building the Ranking Function

def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [22]:
import jpx_tokyo_market_prediction

env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = get_features(prices)
    prices = prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
    prices = prices.fillna(0)
    prices[scaled_features] = scaler.fit_transform(prices[scaled_features])
    pred_1 = model_1.predict(prices[features])
    pred_2 = model_2.predict(prices[features])
    pred_3 = model_3.predict(prices[features])
    dnn_preds = list()
    for model in models:
        dnn_preds.append(model.predict([prices[scaled_features], prices[["SecuritiesCode"]]]))    
    prices['pred'] = np.asarray(list(itertools.chain(*sum(dnn_preds)/len(dnn_preds))))
    prices = add_rank(prices)
    feature_map = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
    env.predict(sample_prediction)

Exception: You can only call `make_env()` once.

In [ ]:
pd.read_csv("./submission.csv")

2022-07-05 03:06:30.584149: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-05 03:06:30.597994: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


InvalidArgumentError:  indices[23,0] = 4423 is not in [0, 4416)
	 [[node model_1/embedding_1/embedding_lookup (defined at tmp/ipykernel_42/3213900137.py:8) ]] [Op:__inference_predict_function_2064]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/embedding_1/embedding_lookup:
 model_1/embedding_1/embedding_lookup/1913 (defined at opt/conda/lib/python3.7/contextlib.py:112)

Function call stack:
predict_function
